In [4]:
#import pandas the data analysis library for python
import pandas as pd 

#upload excel file
df = pd.read_excel("hospital_encounter_data_10_24.xlsx") 
print(df)

       encounter_id  admission_date origin  service_line      loc    drg  \
0      5AP7T95R6HG6           44927    ED   Internal Med  GEN MED  392.0   
1      UBE6BNHXJDV6           44927    ED   Internal Med  GEN MED  871.0   
2      2G6U4ZHPWW5V           44927    ED   Internal Med  GEN MED  871.0   
3      BSHAPPAB3YFC           44927    ED   Internal Med      PCU  871.0   
4      V6XH3DEFDTTT           44927    ED   Internal Med  GEN MED  871.0   
...             ...             ...    ...           ...      ...    ...   
97430  X2TYH9DRWRTF           44926    ED   General Surg  GEN MED  329.0   
97431  XX2DSDWPYSFV           44926     OR  General Surg  GEN MED  620.0   
97432  FT45SDCNHD2T           44926     OR  General Surg       OR  329.0   
97433  JXMF6KDSEQEN           44926     OR  General Surg       OR  620.0   
97434  3CEFCQZSC5K4           44926     OR  General Surg  GEN MED  620.0   

       discharge_date  patient_zipcode  
0               44939            65072  
1    